<img src="https://github.com/rmlarose/qcbq/blob/master/img/banner.png?raw=true" alt="QCBQ Banner">

# Tutorial 4a: Using the Quantum Approximate Optimization Algorithm

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/c/cf/Max-cut.svg/1280px-Max-cut.svg.png" alt="Cartoon graphic of a maximum cut on a weighted graph." style="width:300px;height:200px;">

**Author:** Ryan LaRose

The [QAOA](https://arxiv.org/abs/1411.4028) is a quantum algorithm for approximately solving optimization problems. This notebook will walk you through how to use Qiskit Aqua to solve MaxCut and other combinatorial optimization problems with the QAOA.

## Learning goals

(1) Understand the MaxCut problem and the goal in solving it.

(2) Know how to map an optimization problem (e.g., MaxCut) to a QAOA problem using Qiskit.

(3) Be able to find the optimal parameters for the circuit using optimizers in Qiskit.

(4) Understand how to sample from the QAOA circuit with optimal parameters to obtain approximate solutions.

In [ ]:
"""Imports for the notebook."""
_req = """This notebook is written for

qiskit-aqua==0.6.0

Your code may not execute properly.
"""
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import warnings

import qiskit
if "0.6" not in qiskit.__qiskit_version__.get("qiskit-aqua"):
    warnings.warn(_req)

In [ ]:
"""Optional: Turn off warnings for the notebook."""
warnings.filterwarnings("ignore")

In [ ]:
"""Specific imports for QAOA with MaxCut."""
# Import the QAOA object
from qiskit.aqua.algorithms.adaptive import QAOA

# Import tools for the MaxCut problem
from qiskit.aqua.translators.ising.max_cut import (
    get_max_cut_qubitops, max_cut_value, random_graph
)
from qiskit.aqua.operators.weighted_pauli_operator import (
    Pauli, WeightedPauliOperator
)

# Import optimizers in Qiskit for finding the best parameters in the QAOA circuit
from qiskit.aqua.components.optimizers import ADAM, AQGD, COBYLA, POWELL

In [ ]:
"""Helper function for drawing weighted graphs.

You don't need to know how this function works. You will see how to use it below.
"""
def draw_weighted(graph: nx.Graph,
                  pos_color: str = "blue",
                  neg_color: str = "red",
                  scale: float = 2.0,
                  **kwargs) -> None:
    """Shows a visual of a graph with edges scaled by weight and colored by sign.
    
    Args:
        graph: The weighted graph to visualize.
        pos_color: Color for edges with a positive weight.
        neg_color: Color for edges with a negative weight.
        scale: Floating point value to scale edge weights by
               in the visualization.
            
    Keyword Args:
        cut (List[Int]): A list of 0, 1 values specifying which
                         nodes are in which class. The number of
                         values must be equal to the number of
                         nodes in the graph.
    """
    pos = nx.spring_layout(graph)
    if "cut" in kwargs.keys():
        keys = kwargs["cut"]
        if len(keys) != len(graph.nodes):
            raise ValueError(
                f"ecolor_key has length {len(keys)} but graph has {len(graph.nodes)} nodes."
            )
        nx.draw_networkx_nodes(graph, pos, node_size=700, node_color=keys, cmap=plt.cm.Greens)
    else:
        nx.draw_networkx_nodes(graph, pos, node_size=700)
    
    col = lambda sgn: pos_color if sgn > 0 else neg_color
    
    for edge in graph.edges:
        weight = graph.get_edge_data(*edge)["weight"]
        sgn = np.sign(weight)
        size = abs(weight)
        nx.draw_networkx_edges(graph, 
                               pos, 
                               edgelist=[edge], 
                               width=scale * size,
                               edge_color=col(sgn),
                               alpha=0.5)
    nx.draw_networkx_labels(graph, pos, font_size=20)
    plt.axis("off")
    plt.show()

# Using the QAOA from Qiskit Aqua for MaxCut

## Step 1: Define the problem

### What's MaxCut?

Given a weighted graph $G = (V, E)$ with vertices $V$ and weighted edges $E$, the goal of MaxCut is the following:

Partition the vertices $V$ into two sets such that the quantity $\sum_{e \in C} w(e)$ is maximal. Here, $w(e)$ is the weight of edge $e$, and $C$ is the set of "cut edges," i.e. edges with vertices in different partitioned sets.

Run the cell below to create a random [adjacency matrix](https://en.wikipedia.org/wiki/Adjacency_matrix) representing a weighted graph.

In [ ]:
"""Define the graph for MaxCut via an adjacency matrix."""
nodes = 6  # Vary the number of nodes here
matrix = random_graph(n=nodes, edge_prob=0.5, seed=2)
print("The adjacency matrix is:")
print(matrix)

While more convenient to work with mathematically, adjacency matrices are less aesthetic than graph visualizations. Run the cell below to visualize the graph this matrix represents. The provided function `draw_weighted` colors edges by sign of the weight (positive or negative) and scales the edge width proportional to the magnitude of the weight. 

*Note: Visualizing graphs is a bit of a tricky business. If you get an image that isn't ideal, rerun the cell to get another one.*

In [ ]:
"""Convert the adjacency matrix to a (weighted) graph and visualize it."""
graph = nx.from_numpy_array(matrix, parallel_edges=False)
draw_weighted(graph)

### Try a cut!

Remember: To "win the MaxCut game," we have to divide the vertices in this graph into two sets such that a particular value (described above) is maximal. This value is computed for us below by the function `max_cut_value`. 

**Goal:** Create a partition (cut) by assigning each vertex a value of `0` or `1` via a `numpy.ndarray`. Use the provided function to compute the value for this cut. Update the cut to try and maximize the value.

In [ ]:
"""TODO: Put a list of 1s and 0s to assign vertex sets and see
what the value of your cut is.
"""
cut = np.array([0, 1, 0, 0, 0, 0])  ### <-- Your code here!
print("The value of this cut is:", max_cut_value(cut, matrix))

You can visualize your cut by coloring nodes in the graph two colors. An example of doing so is shown below using the `cut` keyword argument to `draw_weighted`.

In [ ]:
"""Visualize the cut by coloring nodes in distinct vertex sets different colors."""
draw_weighted(graph, cut=[0, 0, 0, 1, 1, 1])

**Question:** How many possible cuts are there for a graph with $n$ nodes?

<font size=8 color="#009600">&#9998;</font> **Answer:** Answer the above question here!

For graphs with a few vertices, we can quickly try all possible cuts and pick the best one. However, for larger graphs, this method is intractable. We need a better method: **the QAOA**!

## Step 2: Translate the problem to quantum

As hinted at above, the objective function for MaxCut on a graph $G = (V, E)$ can be written

\begin{equation}
    H_C = \frac{1}{2} \sum_{(i, j) \in E} w_{ij} (1 - z_i z_j)
\end{equation}

where $w_{ij}$ is the weight of edge $(i, j)$ and

\begin{equation}
    z_i = \begin{cases}
        1 & \text{if node $i$ is in set } A \\
        -1 & \text{if node $i$ is in set } B 
    \end{cases}
\end{equation}

Note that here we use *spins* $\{-1, +1\}$ instead of *bits* $\{0, 1\}$ to label the partitioned sets: The two strategies are of course equivalent.

**Question:** Show that spins and bits are equivalent in the sense that there exists a bijection between the two.

<font size=8 color="#009600">&#9998;</font> **Answer:** Answer the above question here!

**Question:** What is the term $1 - z_i z_j$ if nodes $i$ and $j$ are in the same set? Different sets?

<font size=8 color="#009600">&#9998;</font> **Answer:** Answer the above question here!

The quantum version of the cost function is the exact same except we "promote" the spins to Pauli-$Z$ operators which have eigenvalues in the spin set $\{-1, +1\}$. This makes the cost function a *Hamiltonian* (hermitian matrix) instead of just a *scalar* (number). If you know about Pauli's, convince yourself this is a diagonal Hamiltonian in the computational basis.

In this step, we input an adjacency matrix $W = [w_{ij}]$ describing a weighted graph and output a set of Pauli operators and constant shift which defines the Hamiltonian for the MaxCut cost function.

In [ ]:
"""Pauli operators from matrix."""
op, shift = get_max_cut_qubitops(matrix)

In [ ]:
"""Inspect the Pauli operators."""
print("Edge set of the graph:")
for edge in graph.edges:
    print("Weight:", graph.get_edge_data(*edge)["weight"], "Edge:", edge)
print("\nWeighted Pauli operators.")
for pauli in op.paulis:
    print(2 * np.real(pauli[0]), "*", pauli[1].to_label()[::-1])

**Question:** Inspect the output of the above cell. What is the relationship between the weighted edges and the weighted Pauli operators?

<font size=8 color="#009600">&#9998;</font> **Answer:** Answer the above question here!

## Step 3: Define the QAOA circuit(s)

Here, we use our Pauli operator from above to make the QAOA instance. We also input a classical optimizer which we will use later, and specify the number of layers $p$ in the QAOA circuit.

In [ ]:
"""Make the QAOA instance."""
qaoa = QAOA(op, POWELL(), p=1)

We can see all the settings and parameters of our QAOA object by doing the following.

In [ ]:
"""See the settings of the QAOA object."""
print(qaoa.print_settings())

In [ ]:
"""Inspect the circuits."""
backend = qiskit.BasicAer.get_backend("qasm_simulator")
circs = qaoa.construct_circuit([1, 2], backend=backend)

print(f"There are {len(circs)} circuits.")
print(circs[0])

**Question:** What is the relationship between the number of circuits and the number of Pauli operators/edges in the graph?

<font size=8 color="#009600">&#9998;</font> **Answer:** Answer the above question here!

### Visualization: Sweep the leg (parameters)

Before running the optimizer to find the best angles, we can select a grid of angles $\gamma \times \beta$ to evaluate the cost on and visualize the landscape. (Note this can also be interpretted as a grid search, a type of optimization.)

In [ ]:
"""Set the number of points N to define the grid. Larger N ==> longer runtime."""
N = 10
gammas = np.linspace(-np.pi, np.pi, N)
betas = np.linspace(-np.pi, np.pi, N)

In [ ]:
"""Minor hacks for the QAOA instance to make the grid search possible.
Run this cell without too much thought -- this is necessary because the way Aqua is set up.
"""
quantum_instance = qiskit.aqua.QuantumInstance(backend=qiskit.BasicAer.get_backend("qasm_simulator"))
qaoa._quantum_instance = quantum_instance
qaoa._use_simulator_operator_mode = True

In [ ]:
"""Do the grid search and display the progress."""
import progressbar
bar = progressbar.ProgressBar(maxval=N**2)

costs = np.zeros((len(gammas), len(betas)), dtype=float)
bar.start()
for (ii, gamma) in enumerate(gammas):
    for (jj, beta) in enumerate(betas):
        costs[ii][jj] = qaoa._energy_evaluation(np.array([gamma, beta]))
        bar.update(N * ii + jj)
bar.finish()

Now that we have computed the cost at each point in the grid, we can visualize the landscape. What do you notice about landscape below? Are there any symmetries? Should there be?

In [ ]:
"""Visualize the landscape."""
plt.figure(figsize=(7, 7));
plt.imshow(costs, origin=(0, 0));
plt.xlabel("Gammas")
plt.ylabel("Betas")
plt.colorbar();

**Question:** What is the minimum value of the cost in the above landscape? At which parameter values do they occur?

<font size=8 color="#009600">&#9998;</font> **Answer:** Write some code to answer the above question.

In [ ]:
"""Your code here!"""


In [ ]:
"""Write code to answer the above question here."""
print("Min cost =", np.min(costs))  # Your code here!

## Step 4: Run the optimizer and parse the output

Here, we run the optimizer to get the best angles: i.e., the angles which produce the lowest cost value. This will often produce a better minimum cost than a coarse grid search, and executes faster.

In [ ]:
"""Get a quantum instance and run the algorithm."""
qaoa._optimizer = POWELL()
result = qaoa.run(quantum_instance)

All the results from running QAOA are stored in the `dict` above. We can inspect this manually, or use getter functions to view the optimal cost.

In [ ]:
"""View the optimal cost."""
qaoa.get_optimal_cost()

**Question:** What is the minimum value of the cost in the above landscape? At which parameter values do they occur?

<font size=8 color="#009600">&#9998;</font> **Answer:** Answer the above question here!

## Step 5: Sample from the circuit with optimal parameters

Now that we have found (approximately) optimal parameters for the circuit, we can sample from the circuit with these parameters. When we sample from the circuit, we sample bit strings $\{0, 1\}^n$ (where $n$ is the number of nodes) which tell us (approximately) optimal cuts. The most-frequently measured bit strings produce the lowest cost; bit strings measured very few times produced non-optimal cuts.

In [ ]:
"""Get the circuit with optimal parameters."""
circ = qaoa.get_optimal_circuit()
qreg = circ.qregs[0]
creg = qiskit.ClassicalRegister(6)
circ.add_register(creg)
circ.measure(qreg, creg)
print(circ)

In [ ]:
"""Execute the circuit to sample from it."""
job = qiskit.execute(circ, backend=backend, shots=100000)
res = job.result()
counts = res.get_counts()

In [ ]:
"""Visualize the statistics."""
qiskit.visualization.plot_histogram(counts, figsize=(17, 6))

**Question:** Visually, what are a few of the top-sampled bit strings from the above graph?

<font size=8 color="#009600">&#9998;</font> **Answer:** Answer the above question here!

In [ ]:
"""Get the top sampled bit strings."""
import operator
ntop = 10
top_cuts = sorted(counts, key=operator.itemgetter(1))
print(f"Top {ntop} sampled cuts.")
for cut in top_cuts[:ntop]:
    print(cut[::-1])

### Do your top sampled bit strings provide a good cut for the graph?

In [ ]:
"""Select a sampled cut and see its value."""
cut = np.array([1, 1, 0, 0, 0, 1])  ### <-- Your answer here!
print("The value of this cut is:", max_cut_value(cut, matrix))

Note, if the cut does not provide an (approximately) optimal value, try re-running the optimizer. If this doesn't work, you may need to increase the number of layers of QAOA (see questions/exercises below). Also, you can try looking at more sampled bit strings.

## Brute force search for maximum cut

If there are a small number of possible cuts for the graph, we can quickly enumerate through all of them and store the maximum cut.

In [ ]:
"""Brute force search for the maximum cut."""
import itertools

high = -np.inf
conf = np.zeros(nodes)

cuts = itertools.product(*[[0, 1]] * nodes)

for cut in cuts:
    cur = max_cut_value(np.array(cut), matrix)
    if cur > high:
        conf = np.array(cut)
        high = cur

print("Value of maximum cut:", high)
print("Optimal cut:", conf)

You can visualize the graph again here to check if the answer makes sense!

In [ ]:
"""Visualize the graph to see the maximum cut."""
draw_weighted(graph, cut=conf)

# Questions and exercises

### Try a different mixer

The QAOA circuit evolves with the cost Hamiltonian $H_C$ for some time $\gamma$ and then with a **mixer Hamiltonian** (also called a driver Hamiltonian) $H_M$ for some time $\beta$. At a very high level, the cost Hamiltonian gives information for how to move through the solution space, and the mixer Hamiltonian introduces interference terms. The conventional mixer unitary is an $X$ rotation on each qubit by the same angle $\beta$:

\begin{equation}
    U(\beta) = e^{-i H_M \beta} = \prod_{i = 1}^{n} X_i (\beta)
\end{equation}

Other mixers may give better or worse results. Try experimenting with the `mixer` argument in the Aqua `QAOA` object to see how results vary. You may find [this link](https://github.com/Qiskit/qiskit-aqua/blob/master/qiskit/aqua/algorithms/adaptive/qaoa/qaoa.py) or [this link](https://github.com/Qiskit/qiskit-aqua/blob/ea64d77639929d5b13bdae664d0155f1b9fcfd16/qiskit/aqua/algorithms/adaptive/qaoa/var_form.py#L28) helpful.

### How do different optimizers compare?

We used the `POWELL` optimizer above, but there are others in `aqua.components.optimizers`. Pick a few of these and compare their performance. (How will you compare performance?)

### Increasing the number of QAOA layers

Can you get a lower cost by increasing the number of layers (the `p` parameter in QAOA)?

How does this answer compare to the classical brute force solution?

### Try another problem!

We used QAOA for the MaxCut problem above, but you can use it for many other combinatorial optimization problems. Pick one (in Qiskit Aqua, or not) and do so!

#### Congrats! You just successfully used the QAOA to solve a combinatorial optimization problem.

# Further reading and resources

* [Original QAOA paper](https://arxiv.org/abs/1411.4028).
* [QAOA on MaxCut](https://arxiv.org/abs/1811.08419).
* [From QAOA to QAOA](https://arxiv.org/abs/1709.03489).
* [NASA QAOA Tutorial slides](https://ntrs.nasa.gov/archive/nasa/casi.ntrs.nasa.gov/20190001370.pdf).